In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

In [3]:
full = pd.read_csv('../../../full.csv')

C:\Users\weisj\anaconda3\envs\dsi\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (351) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
full.drop(columns  = ['Unnamed: 0', 'Unnamed: 0_x'], inplace = True)
full.columns = full.columns.str.lower()


In [7]:
full.to_csv('full.csv', index =False)

In [41]:
covid = full.copy()
covid.drop(columns  = ['hazard score', 'wildfire', 'tornado', 'landslide',
                      'earthquake', 'drought', 'heatwave', 'avalanche', 'flood', 'hurricane',
                      'estimate!!sex and age!!total population!!65 years and over!!sex ratio (males per 100 females)', '# hosp w/prosth & orthotic serv'], inplace = True)

In [134]:
covid['id'] = covid['id'].astype(str)

In [54]:
covid.rename(columns = {'estimate!!race!!total population': 'population'}, inplace = True)

In [68]:
beds = pd.read_csv('../../../usa-hospital-beds_dataset_usa-hospital-beds.csv')

In [74]:
beds

,X,Y,FID,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_ADDRESS,HQ_ADDRESS1,HQ_CITY,HQ_STATE,HQ_ZIP_CODE,...,CNTY_FIPS,FIPS,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE
0,-112.066157,33.495498,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,...,13.0,4013.0,62.0,62.0,0,0,NaN,NaN,0,0.0
1,-110.965885,32.181263,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,...,19.0,4019.0,295.0,295.0,2,2,NaN,NaN,0,2.0
2,-119.779742,36.773323,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,...,19.0,6019.0,54.0,54.0,2,2,NaN,NaN,0,2.0
3,-72.957610,41.284400,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,...,9.0,9009.0,216.0,216.0,1,1,NaN,NaN,0,2.0
4,-75.606533,39.740206,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,...,3.0,10003.0,62.0,62.0,0,0,NaN,NaN,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6616,-118.268574,33.726417,6617,USNS Mercy Navy Ship Field Hospital (Temporari...,Department of Defense Hospital,1001 S Seaside Ave,NaN,San Pedro,CA,90731,...,37.0,6037.0,NaN,NaN,0,0,NaN,NaN,0,NaN
6617,-83.502688,42.489142,6618,Suburban Collection Showplace Field Hospital (...,Short Term Acute Care Hospital,46100 Grand River Ave,NaN,Novi,MI,48374,...,125.0,26125.0,NaN,NaN,0,0,NaN,NaN,0,NaN
6618,-112.381230,33.634437,6619,Avenir Behavioral Health Center,Psychiatric Hospital,16561 N Parkview Pl,NaN,Surprise,AZ,85374,...,13.0,4013.0,NaN,NaN,0,0,NaN,NaN,0,NaN
6619,-97.164399,31.487394,6620,Oceans Behavioral Hospital Waco,Psychiatric Hospital,5931 Crosslake Pkwy,NaN,Waco,TX,76712,...,309.0,48309.0,NaN,NaN,0,0,NaN,NaN,0,NaN


In [120]:
beds = beds[(beds['HQ_STATE'] != 'HI') & (beds['HQ_STATE'] != 'AK') & (beds['HQ_STATE'] != 'PR')
           & (beds['HQ_STATE'] != 'GU') & (beds['HQ_STATE'] != 'MP') & (beds['HQ_STATE'] != 'AS')
           & (beds['HQ_STATE'] != 'VI')]
gbeds = beds.groupby('FIPS')['NUM_LICENSED_BEDS'].sum()

In [124]:
gbeds = pd.DataFrame(gbeds)
gbeds['id'] = gbeds.index
gbeds['id'] = gbeds['id'].astype(str)

In [125]:
gbeds.reset_index(drop = True)


,NUM_LICENSED_BEDS,id
0,85.0,1001.0
1,386.0,1003.0
2,74.0,1005.0
3,35.0,1007.0
4,25.0,1009.0
...,...,...
2454,115.0,56037.0
2455,48.0,56039.0
2456,225.0,56041.0
2457,18.0,56043.0


In [127]:
gbeds['id'] = gbeds['id'].str.slice(0,-2)


In [128]:
gbeds['id'] = gbeds['id'].str.slice(0,-2)
gbeds

,NUM_LICENSED_BEDS,id
FIPS,,
1001.0,85.0,1001
1003.0,386.0,1003
1005.0,74.0,1005
1007.0,35.0,1007
1009.0,25.0,1009
...,...,...
56037.0,115.0,56037
56039.0,48.0,56039
56041.0,225.0,56041


In [130]:
gbeds.shape

(2459, 2)

In [160]:
new_beds['NUM_LICENSED_BEDS'].fillna(0, inplace = True)

,county_x,state,id,population,estimate!!race!!total population!!one race,estimate!!race!!total population!!two or more races,estimate!!race!!total population!!one race.1,estimate!!race!!total population!!one race!!white,estimate!!race!!total population!!one race!!black or african american,estimate!!race!!total population!!one race!!american indian and alaska native,...,"nurse practitioners, fmle w/npi_y","clin nurs specialist,male w/npi",# nursing facilities,"cert nurse midwvs,ft oth emptyp_y",nursing facilities total beds,nurse practitioners_y,"advpractnurs midwve,male w/npi",st_num,countyfips,NUM_LICENSED_BEDS


In [172]:
new_beds.to_csv('../../../new_beds.csv')

In [163]:
new_beds.rename(columns = {'NUM_LICENSED_BEDS': 'beds'}, inplace = True)

In [164]:
new_beds.keys()

Index(['county_x', 'state', 'id', 'population',
       'estimate!!race!!total population!!one race',
       'estimate!!race!!total population!!two or more races',
       'estimate!!race!!total population!!one race.1',
       'estimate!!race!!total population!!one race!!white',
       'estimate!!race!!total population!!one race!!black or african american',
       'estimate!!race!!total population!!one race!!american indian and alaska native',
       ...
       'nurse practitioners, fmle w/npi_y', 'clin nurs specialist,male w/npi',
       '# nursing facilities', 'cert nurse midwvs,ft oth emptyp_y',
       'nursing facilities total beds', 'nurse practitioners_y',
       'advpractnurs midwve,male w/npi', 'st_num', 'countyfips', 'beds'],
      dtype='object', length=845)

In [171]:
new_beds['case_per_pop'] = new_beds['total_covid']/new_beds['population']
new_beds['beds_per_pop'] = new_beds['beds']/new_beds['population']
new_beds['beds_per_case'] = new_beds['beds']/new_beds['total_covid']


In [170]:
new_beds[['total_covid', 'population', 'case_per_pop']]

,total_covid,population,case_per_pop
0,67,55200,0.001214
1,208,208107,0.000999
2,53,25782,0.002056
3,44,22527,0.001953
4,44,57645,0.000763
...,...,...,...
3103,12,44117,0.000272
3104,67,23059,0.002906
3105,7,20609,0.000340
3106,5,8129,0.000615
